In [ ]:
from vpython import *
import math

In [ ]:
node_list = {"a": [543,323,433,True], "b":[324,530,342,False], "c":[654,392,214,False],"d":[245,250,123,False], "e":[234,432,234,False], "f":[234,123,142,False], "g": [100,200,300,False]}
edge_list = [{"source":"a", "target":"b"}, {"source":"a", "target":"c"}, {"source":"a", "target":"d"}, {"source":"a", "target":"e"}, {"source":"b", "target":"d"}, {"source":"f", "target":"e"}]

natLength = 200
ks = 0.3
kg = 90
maxRepulsion = 300
drawnNodes = []
drawnLines = []
def draw_nodes(list):
    for drawN in list:
        xcoords = node_list[drawN][0]
        ycoords = node_list[drawN][1]
        zcoords = node_list[drawN][2]
        id = sphere(pos=vector(xcoords,ycoords,zcoords),radius=10,color=color.green, make_trail = True)
        drawnNodes.append(drawN)
        drawnNodes.append(id)
def draw_edge(list):
    for drawE in list:
        sourceX = node_list[drawE["source"]][0]
        targetX = node_list[drawE["target"]][0]
        sourceY = node_list[drawE["source"]][1]
        targetY = node_list[drawE["target"]][1]
        sourceZ = node_list[drawE["source"]][2]
        targetZ = node_list[drawE["target"]][2]
        id = arrow(pos = vector(sourceX, sourceY, sourceZ), axis = vector(targetX-sourceX, targetY-sourceY, targetZ-sourceZ), shaftwidth = 5)
        drawnLines.append(id)

def draw(nodes, edges):
    draw_edge(edges)
    draw_nodes(nodes)

In [ ]:
def move(x,y,z,id):
    xPos = node_list[drawnNodes[id]][0]
    yPos = node_list[drawnNodes[id]][1]
    zPos = node_list[drawnNodes[id]][2]
    newxPos = xPos+x
    newyPos = yPos+y
    newzPos = zPos+z
    drawnNodes[id+1].pos=vector(newxPos,newyPos,newzPos)
    node_list[drawnNodes[id]][0] = newxPos
    node_list[drawnNodes[id]][1] = newyPos
    node_list[drawnNodes[id]][2] = newzPos

def move_edges():
    for i in range(len(edge_list)):
        sourcex = node_list[edge_list[i]["source"]][0]
        sourcey = node_list[edge_list[i]["source"]][1]
        sourcez = node_list[edge_list[i]["source"]][2]
        targetx = node_list[edge_list[i]["target"]][0] - node_list[edge_list[i]["source"]][0]
        targety = node_list[edge_list[i]["target"]][1] - node_list[edge_list[i]["source"]][1]
        targetz = node_list[edge_list[i]["target"]][2] - node_list[edge_list[i]["source"]][2]
        drawnLines[i].pos = vector(sourcex,sourcey,sourcez) 
        drawnLines[i].axis = vector(targetx, targety, targetz)

In [ ]:
def hookesLaw(a,b):
    xdist = b[0]-a[0]
    ydist = b[1]-a[1]
    zdist = b[2]-a[2]
    d3 = xdist*xdist+ydist*ydist+zdist*zdist
    d3 = math.sqrt(d3)
    distance = d3-natLength
    if d3 == 0:
        return [0,0,0]
    if d3 != 0:
        force = ks*distance/d3
    return [xdist*force, ydist*force,zdist*force ]

def coulombsLaw(a,b):
    xdist = b[0]-a[0]
    ydist = b[1]-a[1]
    zdist = b[2]-a[2]
    d3 = xdist*xdist+ydist*ydist+zdist*zdist
    d3 = math.sqrt(d3)
    if d3 == 0 or d3 > maxRepulsion:
        return [0,0,0]
    if d3 != 0 and d3 < maxRepulsion:
        force = kg/(d3*d3)
    return [-force*xdist,-force*ydist,-force*zdist]


In [ ]:
def loop():

    while True:
        sleep(0.25)
        for key1 in node_list:
            netForceX = 0
            netForceY = 0
            netForceZ = 0
            if node_list[key1][3] == False:
                for key2 in node_list:
                    if key1 != key2:
                        addedXCForce = coulombsLaw(node_list[key1], node_list[key2])[0]
                        addedYCForce = coulombsLaw(node_list[key1], node_list[key2])[1]
                        addedZCForce = coulombsLaw(node_list[key1], node_list[key2])[2]
                        netForceX = netForceX + addedXCForce
                        netForceY = netForceY + addedYCForce
                        netForceZ = netForceZ + addedZCForce
                for i in range(len(edge_list)):
                    if key1 == edge_list[i]["source"]:
                        addedXHForce = hookesLaw(node_list[key1], node_list[edge_list[i]["target"]])[0]
                        addedYHForce = hookesLaw(node_list[key1], node_list[edge_list[i]["target"]])[1]
                        addedZHForce = hookesLaw(node_list[key1], node_list[edge_list[i]["target"]])[2]
                        netForceX = netForceX + addedXHForce
                        netForceY = netForceY + addedYHForce
                        netForceZ = netForceZ + addedZHForce
                    if key1 == edge_list[i]["target"]:
                        addedXHForce = hookesLaw(node_list[key1], node_list[edge_list[i]["source"]])[0]
                        addedYHForce = hookesLaw(node_list[key1], node_list[edge_list[i]["source"]])[1]
                        addedZHForce = hookesLaw(node_list[key1], node_list[edge_list[i]["source"]])[2]
                        netForceX = netForceX + addedXHForce
                        netForceY = netForceY + addedYHForce
                        netForceZ = netForceZ + addedZHForce
#Draw stuff here
                for idNum in range(len(drawnNodes)):
                    if drawnNodes[idNum] == key1:
                        move(netForceX,netForceY,netForceZ,idNum)
        move_edges()


In [ ]:
draw(node_list,edge_list)
loop()